In [2]:
import pandas as pd

# Menentukan path file di Google Drive
file_path = 'archive/Data_Tanaman_Padi_Sumatera_version_1.csv'

# Membaca file xls
df = pd.read_csv(file_path)
X = df[["Luas Panen", "Curah hujan", "Kelembapan" ,"Suhu rata-rata"]]
y = df['Produksi']
# print(X.head())
# Menampilkan 5 baris pertama dari DataFrame
print(df.head())
print(X.head())

  Provinsi  Tahun   Produksi  Luas Panen  Curah hujan  Kelembapan  \
0     Aceh   1993  1329536.0    323589.0       1627.0       82.00   
1     Aceh   1994  1299699.0    329041.0       1521.0       82.12   
2     Aceh   1995  1382905.0    339253.0       1476.0       82.72   
3     Aceh   1996  1419128.0    348223.0       1557.0       83.00   
4     Aceh   1997  1368074.0    337561.0       1339.0       82.46   

   Suhu rata-rata  
0           26.06  
1           26.92  
2           26.27  
3           26.08  
4           26.31  
   Luas Panen  Curah hujan  Kelembapan  Suhu rata-rata
0    323589.0       1627.0       82.00           26.06
1    329041.0       1521.0       82.12           26.92
2    339253.0       1476.0       82.72           26.27
3    348223.0       1557.0       83.00           26.08
4    337561.0       1339.0       82.46           26.31


Aktivasi relU

In [13]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

# Buat dataset dummy
X = df[["Luas Panen", "Curah hujan", "Kelembapan" ,"Suhu rata-rata"]]
y = df['Produksi']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Buat model MLPRegressor dengan 2 hidden layer (50 neuron per layer)
mlp = MLPRegressor(hidden_layer_sizes=(50, 50), activation='relu', solver='adam', max_iter=1000, random_state=42)
mlp.fit(X_train, y_train)
y_pred = mlp.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'MSE: {mse:.4f}')
print(f'R² Score: {r2:.4f}')


MSE: 130674736327.8858
R² Score: 0.8522


Aktivasi Linear

In [14]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

# Buat dataset dummy
X = df[["Luas Panen", "Curah hujan", "Kelembapan" ,"Suhu rata-rata"]]
y = df['Produksi']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Buat model MLPRegressor dengan 2 hidden layer (50 neuron per layer)
mlp = MLPRegressor(hidden_layer_sizes=(50, 50), activation='identity', solver='adam', max_iter=1000, random_state=42)
mlp.fit(X_train, y_train)
y_pred = mlp.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'MSE: {mse:.4f}')
print(f'R² Score: {r2:.4f}')


MSE: 166000978277.9355
R² Score: 0.8122


In [ ]:
Aktivasi Leaky relU

In [22]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Normalisasi dataset
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df[["Luas Panen", "Curah hujan", "Kelembapan", "Suhu rata-rata"]].values)
y = df['Produksi'].values

# Split data
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Konversi ke tensor PyTorch
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

# Model dengan Leaky ReLU
class MLPLeakyReLU(nn.Module):
    def __init__(self):
        super(MLPLeakyReLU, self).__init__()
        self.fc1 = nn.Linear(4, 50)
        self.fc2 = nn.Linear(50, 50)
        self.fc3 = nn.Linear(50, 1)
        self.leaky_relu = nn.LeakyReLU(negative_slope=0.01)

    def forward(self, x):
        x = self.leaky_relu(self.fc1(x))
        x = self.leaky_relu(self.fc2(x))
        x = self.fc3(x)  # Output layer (linear activation)
        return x

# Inisialisasi model
model_leaky = MLPLeakyReLU()
criterion = nn.MSELoss()
optimizer = optim.Adam(model_leaky.parameters(), lr=0.01)

# Training
epochs = 1000
for epoch in range(epochs):
    optimizer.zero_grad()
    y_pred_train = model_leaky(X_train_tensor)
    loss = criterion(y_pred_train, y_train_tensor)
    loss.backward()
    optimizer.step()

    if epoch % 100 == 0:
        print(f'Epoch {epoch}: Loss = {loss.item():.4f}')

# Evaluasi
y_pred_test = model_leaky(X_test_tensor).detach().numpy()
mse = mean_squared_error(y_test, y_pred_test)
r2 = r2_score(y_test, y_pred_test)

print(f'MSE (Leaky ReLU): {mse:.4f}')
print(f'R² Score (Leaky ReLU): {r2:.4f}')


Epoch 0: Loss = 4434771312640.0000
Epoch 100: Loss = 4406700933120.0000
Epoch 200: Loss = 4089013600256.0000
Epoch 300: Loss = 3185484496896.0000
Epoch 400: Loss = 1900727762944.0000
Epoch 500: Loss = 883781468160.0000
Epoch 600: Loss = 468585709568.0000
Epoch 700: Loss = 377618432000.0000
Epoch 800: Loss = 347238432768.0000
Epoch 900: Loss = 327450263552.0000
MSE (Leaky ReLU): 108650920154.6803
R² Score (Leaky ReLU): 0.8771


Aktifasi Swish

In [23]:
# Model dengan Swish Activation
class MLPSwish(nn.Module):
    def __init__(self):
        super(MLPSwish, self).__init__()
        self.fc1 = nn.Linear(4, 50)
        self.fc2 = nn.Linear(50, 50)
        self.fc3 = nn.Linear(50, 1)

    def swish(self, x):
        return x * torch.sigmoid(x)

    def forward(self, x):
        x = self.swish(self.fc1(x))
        x = self.swish(self.fc2(x))
        x = self.fc3(x)  # Output layer (linear activation)
        return x

# Inisialisasi model
model_swish = MLPSwish()
criterion = nn.MSELoss()
optimizer = optim.Adam(model_swish.parameters(), lr=0.01)

# Training
for epoch in range(epochs):
    optimizer.zero_grad()
    y_pred_train = model_swish(X_train_tensor)
    loss = criterion(y_pred_train, y_train_tensor)
    loss.backward()
    optimizer.step()

    if epoch % 100 == 0:
        print(f'Epoch {epoch}: Loss = {loss.item():.4f}')

# Evaluasi
y_pred_test = model_swish(X_test_tensor).detach().numpy()
mse = mean_squared_error(y_test, y_pred_test)
r2 = r2_score(y_test, y_pred_test)

print(f'MSE (Swish): {mse:.4f}')
print(f'R² Score (Swish): {r2:.4f}')


Epoch 0: Loss = 4434771312640.0000
Epoch 100: Loss = 4410982793216.0000
Epoch 200: Loss = 4144704520192.0000
Epoch 300: Loss = 3376569647104.0000
Epoch 400: Loss = 2190993391616.0000
Epoch 500: Loss = 1111582179328.0000
Epoch 600: Loss = 575222054912.0000
Epoch 700: Loss = 403817988096.0000
Epoch 800: Loss = 344630493184.0000
Epoch 900: Loss = 316441329664.0000
MSE (Swish): 76406624041.7528
R² Score (Swish): 0.9136


Aktivasi GelU

In [24]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score

# Normalisasi dataset
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df[["Luas Panen", "Curah hujan", "Kelembapan", "Suhu rata-rata"]].values)
y = df['Produksi'].values

# Split data
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Konversi ke tensor PyTorch
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

# Model dengan GELU Activation
class MLPGELU(nn.Module):
    def __init__(self):
        super(MLPGELU, self).__init__()
        self.fc1 = nn.Linear(4, 50)
        self.fc2 = nn.Linear(50, 50)
        self.fc3 = nn.Linear(50, 1)
        self.gelu = nn.GELU()  # Fungsi aktivasi GELU

    def forward(self, x):
        x = self.gelu(self.fc1(x))
        x = self.gelu(self.fc2(x))
        x = self.fc3(x)  # Output layer (linear activation)
        return x

# Inisialisasi model
model_gelu = MLPGELU()
criterion = nn.MSELoss()
optimizer = optim.Adam(model_gelu.parameters(), lr=0.01)

# Training
epochs = 1000
for epoch in range(epochs):
    optimizer.zero_grad()
    y_pred_train = model_gelu(X_train_tensor)
    loss = criterion(y_pred_train, y_train_tensor)
    loss.backward()
    optimizer.step()

    if epoch % 100 == 0:
        print(f'Epoch {epoch}: Loss = {loss.item():.4f}')

# Evaluasi
y_pred_test = model_gelu(X_test_tensor).detach().numpy()
mse = mean_squared_error(y_test, y_pred_test)
r2 = r2_score(y_test, y_pred_test)

print(f'MSE (GELU): {mse:.4f}')
print(f'R² Score (GELU): {r2:.4f}')


Epoch 0: Loss = 4434771312640.0000
Epoch 100: Loss = 4409313460224.0000
Epoch 200: Loss = 4126554718208.0000
Epoch 300: Loss = 3319364059136.0000
Epoch 400: Loss = 2117670010880.0000
Epoch 500: Loss = 1062669516800.0000
Epoch 600: Loss = 533405761536.0000
Epoch 700: Loss = 383799754752.0000
Epoch 800: Loss = 341333639168.0000
Epoch 900: Loss = 318662377472.0000
MSE (GELU): 106278699517.6254
R² Score (GELU): 0.8798


Aktivasi Sigmoid

In [25]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

# Buat dataset dummy
X = df[["Luas Panen", "Curah hujan", "Kelembapan" ,"Suhu rata-rata"]]
y = df['Produksi']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Buat model MLPRegressor dengan 2 hidden layer (50 neuron per layer)
mlp = MLPRegressor(hidden_layer_sizes=(50, 50), activation='logistic', solver='adam', max_iter=1000, random_state=42)
mlp.fit(X_train, y_train)
y_pred = mlp.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'MSE: {mse:.4f}')
print(f'R² Score: {r2:.4f}')


MSE: 3087738564988.0225
R² Score: -2.4924


c:\Users\user\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(


Aktivasi Tanh

In [26]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

# Buat dataset dummy
X = df[["Luas Panen", "Curah hujan", "Kelembapan" ,"Suhu rata-rata"]]
y = df['Produksi']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Buat model MLPRegressor dengan 2 hidden layer (50 neuron per layer)
mlp = MLPRegressor(hidden_layer_sizes=(50, 50), activation='tanh', solver='adam', max_iter=1000, random_state=42)
mlp.fit(X_train, y_train)
y_pred = mlp.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'MSE: {mse:.4f}')
print(f'R² Score: {r2:.4f}')


MSE: 3087718145609.8096
R² Score: -2.4924


c:\Users\user\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(


In [14]:
# Mengambil bobot dan bias
weights = mlp.coefs_  # Bobot antar lapisan
biases = mlp.intercepts_  # Bias tiap lapisan

# Tampilkan bobot dan bias
for i, (w, b) in enumerate(zip(weights, biases)):
    print(f"Lapisan {i+1}:")
    print("Bobot:\n", w)
    print("Bias:\n", b)
    print("-" * 30)

Lapisan 1:
Bobot:
 [[-2.29396844e-02  4.58853547e-01  3.29696793e-01  1.99115396e-01
  -1.23667056e-01 -1.23680231e-01 -1.79076384e-01  3.64855806e-01
   1.81798089e-01  2.91845146e-01 -2.01098376e-01  4.54283995e-01
   3.21286439e-01 -9.37457333e-02 -1.09737724e-01 -1.08896505e-01
  -4.98652785e-02  1.92370007e-01 -7.70896040e-03 -5.55881712e-02
   3.70654572e-03 -1.32755605e-01 -5.51793064e-02 -2.56650305e-02
  -3.64663764e-03  2.27215894e-01 -1.00314964e-01  1.81674845e-01
   2.09578160e-01 -1.85868791e-01  2.20765150e-01 -1.15795525e-01
  -1.75026127e-01  4.63198877e-01  4.55699880e-01  2.07450016e-01
  -4.97047706e-02 -1.56262604e-01  2.86777499e-01 -6.16015789e-03
  -1.42484276e-01 -1.40064405e-02 -1.92949729e-01  4.29974825e-01
  -7.06327596e-02  2.75381855e-01 -4.66707084e-02  1.77985918e-01
  -5.06171958e-03 -1.08125525e-01]
 [ 1.95290319e-01  3.43178752e-01  4.69429512e-01  3.98200197e-01
   1.47200825e-02  1.67465340e-01 -1.61508115e-01 -8.02842992e-02
  -1.86710193e-01  3.7